In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
import numpy as np
import random
import scipy
import pickle
import os
import scalebar
import nept

from loading_data import get_data
from analyze_tuning_curves import get_only_tuning_curves
from utils_maze import get_zones, get_bin_centers
from analyze_decode_swrs import bin_spikes, plot_summary_individual, plot_likelihood_overspace, get_likelihood, plot_combined, plot_stacked_summary, get_likelihoods, save_likelihoods, pickle_likelihoods

In [2]:
thisdir = os.getcwd()
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "trials", "decoding", "shuffled")
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [3]:
# Set random seeds
random.seed(0)
np.random.seed(0)

In [4]:
def get_decoded_swr_plots(infos, group):
    plot_individual = False
    update_cache = False

    n_shuffles = 100
    percentile_thresh = 99

    colours = dict()
    colours["u"] = "#2b8cbe"
    colours["shortcut"] = "#31a354"
    colours["novel"] = "#d95f0e"
    colours["other"] = "#bdbdbd"

    # swr params
    z_thresh = 2.0
    power_thresh = 3.0
    merge_thresh = 0.02
    min_length = 0.05
    swr_thresh = (140.0, 250.0)

    task_times = ["prerecord", "pauseA", "pauseB", "postrecord"]
    maze_segments = ["u", "shortcut", "novel", "other"]

    n_sessions = len(infos)
    all_likelihoods_true = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
    all_likelihoods_shuff = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
    all_likelihoods_proportion = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
    all_likelihoods_true_passthresh = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
    all_likelihoods_true_passthresh_n_swr = {task_time: 0 for task_time in task_times}
    all_compareshuffle = {task_time: {trajectory: 0 for trajectory in maze_segments} for task_time in task_times}

    n_all_swrs = {task_time: 0 for task_time in task_times}

    for info in infos:
        print(info.session_id)

        events, position, spikes, lfp, _ = get_data(info)

        # Define zones
        zones = dict()
        zones["u"], zones["shortcut"], zones["novel"] = get_zones(info, position, subset=True)
        combined_zones = zones["u"] + zones["shortcut"] + zones["novel"]
        zones["other"] = ~combined_zones

        # Find SWRs for the whole session
        swrs_path = os.path.join(pickle_filepath, info.session_id+"_swrs.pkl")

        # Remove previous pickle if update_cache
        if update_cache:
            if os.path.exists(swrs_path):
                os.remove(swrs_path)

        # Load pickle if it exists, otherwise compute and pickle
        if os.path.exists(swrs_path):
            print("Loading pickled true likelihoods...")
            with open(swrs_path, 'rb') as fileobj:
                swrs = pickle.load(fileobj)
        else:
            swrs = nept.detect_swr_hilbert(lfp, fs=info.fs, thresh=swr_thresh, z_thresh=z_thresh,
                                           power_thresh=power_thresh, merge_thresh=merge_thresh, min_length=min_length)
            swrs = nept.find_multi_in_epochs(spikes, swrs, min_involved=4)

        rest_epochs = nept.rest_threshold(position, thresh=12., t_smooth=0.8)

        # Restrict SWRs to those during epochs of interest during rest
        phase_swrs = dict()
        n_swrs = {task_time: 0 for task_time in task_times}

        for task_time in task_times:
            epochs_of_interest = info.task_times[task_time].intersect(rest_epochs)

            phase_swrs[task_time] = epochs_of_interest.overlaps(swrs)
            phase_swrs[task_time] = phase_swrs[task_time][phase_swrs[task_time].durations >= 0.05]

            n_swrs[task_time] += phase_swrs[task_time].n_epochs
            n_all_swrs[task_time] += phase_swrs[task_time].n_epochs

        raw_path_true = os.path.join(pickle_filepath, info.session_id+"_raw-likelihoods_true.pkl")
        sum_path_true = os.path.join(pickle_filepath, info.session_id+"_sum-likelihoods_true.pkl")

        # Remove previous pickle if update_cache
        if update_cache:
            if os.path.exists(raw_path_true):
                os.remove(raw_path_true)
            if os.path.exists(sum_path_true):
                os.remove(sum_path_true)

        compute_likelihoods = False

        # Load pickle if it exists, otherwise compute and pickle
        if os.path.exists(raw_path_true) and os.path.exists(sum_path_true):
            print("Loading pickled true likelihoods...")
            with open(raw_path_true, 'rb') as fileobj:
                raw_likelihoods_true = pickle.load(fileobj)
            with open(sum_path_true, 'rb') as fileobj:
                session_sums_true = pickle.load(fileobj)

        combined_likelihoods_shuff = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
        raw_likelihoods_shuffs = {task_time: [] for task_time in task_times}

        for i_shuffle in range(n_shuffles):
            raw_path_shuff = os.path.join(pickle_filepath,
                                          info.session_id+"_raw-likelihoods_shuffled-%03d.pkl" % i_shuffle)
            sum_path_shuff = os.path.join(pickle_filepath,
                                          info.session_id+"_sum-likelihoods_shuffled-%03d.pkl" % i_shuffle)

            # Remove previous pickle if update_cache
            if update_cache:
                if os.path.exists(raw_path_shuff):
                    os.remove(raw_path_shuff)
                if os.path.exists(sum_path_shuff):
                    os.remove(sum_path_shuff)

            # Load pickle if it exists, otherwise compute and pickle
            if os.path.exists(raw_path_shuff) and os.path.exists(sum_path_shuff):
                print("Loading pickled shuffled likelihoods "+str(i_shuffle)+"...")
                with open(raw_path_shuff, 'rb') as fileobj:
                    raw_likelihoods_shuff = pickle.load(fileobj)
                with open(sum_path_shuff, 'rb') as fileobj:
                    session_sums_shuff = pickle.load(fileobj)
            else:
                compute_likelihoods = True
                break

            for task_time in task_times:
                raw_likelihoods_shuffs[task_time].append(raw_likelihoods_shuff[task_time])
                for trajectory in maze_segments:
                    combined_likelihoods_shuff[task_time][trajectory].append(np.array(session_sums_shuff[task_time][trajectory]))
        else:
            compute_likelihoods = True

        if compute_likelihoods:
            session_sums_true, raw_likelihoods_true, combined_likelihoods_shuff, raw_likelihoods_shuffs = save_likelihoods(info, position, spikes, phase_swrs, zones, task_times, maze_segments, n_shuffles)

        compareshuffle = {task_time: {trajectory: 0 for trajectory in maze_segments} for task_time in task_times}
        percentiles = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
        passedshuffthresh = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}

        keep_idx = {task_time: [] for task_time in task_times}

        for task_time in task_times:
            raw_likelihoods_shuffs[task_time] = np.swapaxes(raw_likelihoods_shuffs[task_time], 0, 1)
            for trajectory in maze_segments:
                for idx, event in enumerate(range(len(session_sums_true[task_time][trajectory]))):
                    percentile = scipy.stats.percentileofscore(np.sort(np.array(combined_likelihoods_shuff[task_time][trajectory])[:,event]),
                                                               session_sums_true[task_time][trajectory][event])
                    percentiles[task_time][trajectory].append(percentile)
                    if percentile >= percentile_thresh:
                        compareshuffle[task_time][trajectory] += 1
                        all_compareshuffle[task_time][trajectory] += 1
                        keep_idx[task_time].append(idx)

        morelikelythanshuffle_proportion = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
        mean_combined_likelihoods_shuff = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
        passedshuffthresh_n_swr = {task_time: 0 for task_time in task_times}

        for task_time in task_times:
            passedshuffthresh_n_swr[task_time] += len(np.unique(keep_idx[task_time]))
            all_likelihoods_true_passthresh_n_swr[task_time] += len(np.unique(keep_idx[task_time]))
            for trajectory in maze_segments:
                if len(np.sort(np.unique(keep_idx[task_time]))) > 0:
                    passedshuffthresh[task_time][trajectory].append(np.array(session_sums_true[task_time][trajectory])[np.sort(np.unique(keep_idx[task_time]))])
                if len(session_sums_true[task_time][trajectory]) > 0:
                    morelikelythanshuffle_proportion[task_time][trajectory].append(compareshuffle[task_time][trajectory] / len(session_sums_true[task_time][trajectory]))
                else:
                    morelikelythanshuffle_proportion[task_time][trajectory].append(0.0)
                mean_combined_likelihoods_shuff[task_time][trajectory] = np.nanmean(combined_likelihoods_shuff[task_time][trajectory], axis=0)

                all_likelihoods_true[task_time][trajectory].extend(session_sums_true[task_time][trajectory])
                if len(passedshuffthresh[task_time][trajectory]) > 0:
                    all_likelihoods_true_passthresh[task_time][trajectory].append(passedshuffthresh[task_time][trajectory][0])
                else:
                    all_likelihoods_true_passthresh[task_time][trajectory].append([])
                all_likelihoods_shuff[task_time][trajectory].extend(mean_combined_likelihoods_shuff[task_time][trajectory])
                all_likelihoods_proportion[task_time][trajectory].extend(morelikelythanshuffle_proportion[task_time][trajectory])

                if plot_individual:
                    # plot percentiles
                    fig, ax = plt.subplots()
                    n = np.arange(len(percentiles[task_time][trajectory]))
                    plt.bar(n, np.sort(percentiles[task_time][trajectory]), color=colours[trajectory])
                    ax.axhline(percentile_thresh, ls="--", lw=1.5, color="k")
                    title = info.session_id + " individual SWR percentile with shuffle" + str(n_shuffles) + " for " + task_time + " " + trajectory
                    plt.title(title, fontsize=11)
                    plt.tight_layout()
                    plt.savefig(os.path.join(output_filepath, "percentiles", title))
                    plt.close()

            filepath = os.path.join(output_filepath, info.session_id+"-average-likelihood-overspace_"+task_time+".png")
            if len(session_sums_true[task_time]) > 0:
                if plot_individual:
                    plot_likelihood_overspace(info, position, raw_likelihoods_true[task_time],
                                              zones, colours, filepath)

        filename = info.session_id + " proportion of SWRs above "+str(percentile_thresh)+" percentile"
        plot_combined(morelikelythanshuffle_proportion, passedshuffthresh_n_swr,
                      task_times, maze_segments, n_sessions=1, colours=colours, filename=filename)

        filename = info.session_id + " average posteriors during SWRs_sum-shuffled"+str(n_shuffles)
        plot_combined(mean_combined_likelihoods_shuff, n_swrs, task_times, maze_segments,
                      n_sessions=1, colours=colours, filename=filename)

        filename = info.session_id + " average posteriors during SWRs_sum-true"
        plot_combined(session_sums_true, n_swrs, task_times, maze_segments,
                      n_sessions=1, colours=colours, filename=filename)

        filename = info.session_id + " average posteriors during SWRs_sum-true_passthresh"
        plot_combined(passedshuffthresh, n_swrs, task_times, maze_segments,
                      n_sessions=1, colours=colours, filename=filename)

        if plot_individual:
            for task_time in task_times:
                for idx in range(phase_swrs[task_time].n_epochs):
                    filename = info.session_id + "_" + task_time + "_summary-swr" + str(idx) + ".png"
                    filepath = os.path.join(output_filepath, "swr", filename)
                    plot_summary_individual(info, raw_likelihoods_true[task_time][idx],
                                            raw_likelihoods_shuffs[task_time][idx],
                                            position, lfp, spikes,
                                            phase_swrs[task_time].starts[idx],
                                            phase_swrs[task_time].stops[idx],
                                            zones, maze_segments, colours, filepath, savefig=True)

    n_total = {task_time: 0 for task_time in task_times}
    for task_time in task_times:
        for trajectory in maze_segments:
            n_total[task_time] += all_compareshuffle[task_time][trajectory]

    all_compareshuffles = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
    for task_time in task_times:
        for trajectory in maze_segments:
            all_compareshuffles[task_time][trajectory].append(all_compareshuffle[task_time][trajectory] / n_total[task_time])


    for task_time in task_times:
        for trajectory in maze_segments:   
            all_likelihoods_true_passthresh[task_time][trajectory] = np.atleast_1d(np.squeeze(np.hstack(all_likelihoods_true_passthresh[task_time][trajectory])))



    filename = group + " average posteriors during SWRs_sum-shuffled"+str(n_shuffles)
    plot_combined(all_likelihoods_shuff, n_all_swrs, task_times, maze_segments,
                  n_sessions=len(infos), colours=colours, filename=filename)

    filename = group + " average posteriors during SWRs_sum-true"
    plot_combined(all_likelihoods_true, n_all_swrs, task_times, maze_segments,
                  n_sessions=len(infos), colours=colours, filename=filename)

    filename = group + " average posteriors during SWRs_sum-true_passthresh"
    plot_combined(all_likelihoods_true_passthresh, all_likelihoods_true_passthresh_n_swr,
                  task_times, maze_segments, n_sessions=len(infos), colours=colours, filename=filename)

    filename = group + " average posteriors during SWRs_sum-true_passthresh-overallproportion"
    plot_combined(all_compareshuffles, all_likelihoods_true_passthresh_n_swr,
                  task_times, maze_segments, n_sessions=len(infos), colours=colours, filename=filename)

    filename = group + "average posteriors during SWRs_sum-stacked-shuffled"+str(n_shuffles)
    plot_stacked_summary(all_likelihoods_shuff, n_all_swrs, task_times, maze_segments,
                         n_sessions=len(infos), colours=colours, filename=filename)

    filename = group + " average posteriors during SWRs_sum-stacked-true"
    plot_stacked_summary(all_likelihoods_true, n_all_swrs, task_times, maze_segments,
                         n_sessions=len(infos), colours=colours, filename=filename)

    filename = group + " average posteriors during SWRs_sum-stacked-true_passthresh"
    plot_stacked_summary(all_likelihoods_true_passthresh, n_all_swrs, task_times,
                         maze_segments, n_sessions=len(infos), colours=colours, filename=filename)

    filename = group + " proportion of SWRs above the "+str(percentile_thresh)+" percentile (shuffle" + str(n_shuffles) + ")"
    plot_combined(all_likelihoods_proportion, n_all_swrs, task_times, maze_segments,
                  n_sessions=len(infos), colours=colours, filename=filename)


In [5]:
import info.r063d2 as r063d2
import info.r068d8 as r068d8
# infos = [r068d8, r063d2]
# group = "test"
from run import (analysis_infos,
                 r063_infos, r066_infos, r067_infos, r068_infos,
                 days1234_infos, days5678_infos,
                 day1_infos, day2_infos, day3_infos, day4_infos, day5_infos, day6_infos, day7_infos, day8_infos)
# infos = analysis_infos
# group = "All"

# infos = r068_infos
# group = "R068"

In [ ]:
get_decoded_swr_plots(analysis_infos, "All")
get_decoded_swr_plots(r063_infos, "R063")
get_decoded_swr_plots(r066_infos, "R066")
get_decoded_swr_plots(r067_infos, "R067")
get_decoded_swr_plots(r068_infos, "R068")
get_decoded_swr_plots(days1234_infos, "Days1234")
get_decoded_swr_plots(days5678_infos, "Days5678")
get_decoded_swr_plots(day1_infos, "Day1")
get_decoded_swr_plots(day2_infos, "Day2")
get_decoded_swr_plots(day3_infos, "Day3")
get_decoded_swr_plots(day4_infos, "Day4")
get_decoded_swr_plots(day5_infos, "Day5")
get_decoded_swr_plots(day6_infos, "Day6")
get_decoded_swr_plots(day7_infos, "Day7")
get_decoded_swr_plots(day8_infos, "Day8")

R063d2
Loading pickled true likelihoods...
Loading pickled shuffled likelihoods 0...
Loading pickled shuffled likelihoods 1...
Loading pickled shuffled likelihoods 2...
Loading pickled shuffled likelihoods 3...
Loading pickled shuffled likelihoods 4...
Loading pickled shuffled likelihoods 5...
Loading pickled shuffled likelihoods 6...
Loading pickled shuffled likelihoods 7...
Loading pickled shuffled likelihoods 8...
Loading pickled shuffled likelihoods 9...
Loading pickled shuffled likelihoods 10...
Loading pickled shuffled likelihoods 11...
Loading pickled shuffled likelihoods 12...
Loading pickled shuffled likelihoods 13...
Loading pickled shuffled likelihoods 14...
Loading pickled shuffled likelihoods 15...
Loading pickled shuffled likelihoods 16...
Loading pickled shuffled likelihoods 17...
Loading pickled shuffled likelihoods 18...
Loading pickled shuffled likelihoods 19...
Loading pickled shuffled likelihoods 20...
Loading pickled shuffled likelihoods 21...
Loading pickled shuff

Loading pickled shuffled likelihoods 91...
Loading pickled shuffled likelihoods 92...
Loading pickled shuffled likelihoods 93...
Loading pickled shuffled likelihoods 94...
Loading pickled shuffled likelihoods 95...
Loading pickled shuffled likelihoods 96...
Loading pickled shuffled likelihoods 97...
Loading pickled shuffled likelihoods 98...
Loading pickled shuffled likelihoods 99...
R063d4
Loading pickled true likelihoods...
Loading pickled shuffled likelihoods 0...
Loading pickled shuffled likelihoods 1...
Loading pickled shuffled likelihoods 2...
Loading pickled shuffled likelihoods 3...
Loading pickled shuffled likelihoods 4...
Loading pickled shuffled likelihoods 5...
Loading pickled shuffled likelihoods 6...
Loading pickled shuffled likelihoods 7...
Loading pickled shuffled likelihoods 8...
Loading pickled shuffled likelihoods 9...
Loading pickled shuffled likelihoods 10...
Loading pickled shuffled likelihoods 11...
Loading pickled shuffled likelihoods 12...
Loading pickled shuff

Loading pickled shuffled likelihoods 81...
Loading pickled shuffled likelihoods 82...
Loading pickled shuffled likelihoods 83...
Loading pickled shuffled likelihoods 84...
Loading pickled shuffled likelihoods 85...
Loading pickled shuffled likelihoods 86...
Loading pickled shuffled likelihoods 87...
Loading pickled shuffled likelihoods 88...
Loading pickled shuffled likelihoods 89...
Loading pickled shuffled likelihoods 90...
Loading pickled shuffled likelihoods 91...
Loading pickled shuffled likelihoods 92...
Loading pickled shuffled likelihoods 93...
Loading pickled shuffled likelihoods 94...
Loading pickled shuffled likelihoods 95...
Loading pickled shuffled likelihoods 96...
Loading pickled shuffled likelihoods 97...
Loading pickled shuffled likelihoods 98...
Loading pickled shuffled likelihoods 99...
R063d6
Loading pickled true likelihoods...
Loading pickled shuffled likelihoods 0...
Loading pickled shuffled likelihoods 1...
Loading pickled shuffled likelihoods 2...
Loading pickle

Loading pickled shuffled likelihoods 70...
Loading pickled shuffled likelihoods 71...
Loading pickled shuffled likelihoods 72...
Loading pickled shuffled likelihoods 73...
Loading pickled shuffled likelihoods 74...
Loading pickled shuffled likelihoods 75...
Loading pickled shuffled likelihoods 76...
Loading pickled shuffled likelihoods 77...
Loading pickled shuffled likelihoods 78...
Loading pickled shuffled likelihoods 79...
Loading pickled shuffled likelihoods 80...
Loading pickled shuffled likelihoods 81...
Loading pickled shuffled likelihoods 82...
Loading pickled shuffled likelihoods 83...
Loading pickled shuffled likelihoods 84...
Loading pickled shuffled likelihoods 85...
Loading pickled shuffled likelihoods 86...
Loading pickled shuffled likelihoods 87...
Loading pickled shuffled likelihoods 88...
Loading pickled shuffled likelihoods 89...
Loading pickled shuffled likelihoods 90...
Loading pickled shuffled likelihoods 91...
Loading pickled shuffled likelihoods 92...
Loading pic

Loading pickled shuffled likelihoods 57...
Loading pickled shuffled likelihoods 58...
Loading pickled shuffled likelihoods 59...
Loading pickled shuffled likelihoods 60...
Loading pickled shuffled likelihoods 61...
Loading pickled shuffled likelihoods 62...
Loading pickled shuffled likelihoods 63...
Loading pickled shuffled likelihoods 64...
Loading pickled shuffled likelihoods 65...
Loading pickled shuffled likelihoods 66...
Loading pickled shuffled likelihoods 67...
Loading pickled shuffled likelihoods 68...
Loading pickled shuffled likelihoods 69...
Loading pickled shuffled likelihoods 70...
Loading pickled shuffled likelihoods 71...
Loading pickled shuffled likelihoods 72...
Loading pickled shuffled likelihoods 73...
Loading pickled shuffled likelihoods 74...
Loading pickled shuffled likelihoods 75...
Loading pickled shuffled likelihoods 76...
Loading pickled shuffled likelihoods 77...
Loading pickled shuffled likelihoods 78...
Loading pickled shuffled likelihoods 79...
Loading pic

...

Loading pickled shuffled likelihoods 43...
Loading pickled shuffled likelihoods 44...
Loading pickled shuffled likelihoods 45...
Loading pickled shuffled likelihoods 46...
Loading pickled shuffled likelihoods 47...
Loading pickled shuffled likelihoods 48...
Loading pickled shuffled likelihoods 49...
Loading pickled shuffled likelihoods 50...
Loading pickled shuffled likelihoods 51...
Loading pickled shuffled likelihoods 52...
Loading pickled shuffled likelihoods 53...
Loading pickled shuffled likelihoods 54...
Loading pickled shuffled likelihoods 55...
Loading pickled shuffled likelihoods 56...
Loading pickled shuffled likelihoods 57...
Loading pickled shuffled likelihoods 58...
Loading pickled shuffled likelihoods 59...
Loading pickled shuffled likelihoods 60...
Loading pickled shuffled likelihoods 61...
Loading pickled shuffled likelihoods 62...
Loading pickled shuffled likelihoods 63...
Loading pickled shuffled likelihoods 64...
Loading pickled shuffled likelihoods 65...
Loading pic

Loading pickled shuffled likelihoods 35...
Loading pickled shuffled likelihoods 36...
Loading pickled shuffled likelihoods 37...
Loading pickled shuffled likelihoods 38...
Loading pickled shuffled likelihoods 39...
Loading pickled shuffled likelihoods 40...
Loading pickled shuffled likelihoods 41...
Loading pickled shuffled likelihoods 42...
Loading pickled shuffled likelihoods 43...
Loading pickled shuffled likelihoods 44...
Loading pickled shuffled likelihoods 45...
Loading pickled shuffled likelihoods 46...
Loading pickled shuffled likelihoods 47...
Loading pickled shuffled likelihoods 48...
Loading pickled shuffled likelihoods 49...
Loading pickled shuffled likelihoods 50...
Loading pickled shuffled likelihoods 51...
Loading pickled shuffled likelihoods 52...
Loading pickled shuffled likelihoods 53...
Loading pickled shuffled likelihoods 54...
Loading pickled shuffled likelihoods 55...
Loading pickled shuffled likelihoods 56...
Loading pickled shuffled likelihoods 57...
Loading pic

Loading pickled shuffled likelihoods 42...
Loading pickled shuffled likelihoods 43...
Loading pickled shuffled likelihoods 44...
Loading pickled shuffled likelihoods 45...
Loading pickled shuffled likelihoods 46...
Loading pickled shuffled likelihoods 47...
Loading pickled shuffled likelihoods 48...
Loading pickled shuffled likelihoods 49...
Loading pickled shuffled likelihoods 50...
Loading pickled shuffled likelihoods 51...
Loading pickled shuffled likelihoods 52...
Loading pickled shuffled likelihoods 53...
Loading pickled shuffled likelihoods 54...
Loading pickled shuffled likelihoods 55...
Loading pickled shuffled likelihoods 56...
Loading pickled shuffled likelihoods 57...
Loading pickled shuffled likelihoods 58...
Loading pickled shuffled likelihoods 59...
Loading pickled shuffled likelihoods 60...
Loading pickled shuffled likelihoods 61...
Loading pickled shuffled likelihoods 62...
Loading pickled shuffled likelihoods 63...
Loading pickled shuffled likelihoods 64...
Loading pic

Loading pickled shuffled likelihoods 97...
Loading pickled shuffled likelihoods 98...
Loading pickled shuffled likelihoods 99...
R067d3
Loading pickled true likelihoods...
Loading pickled shuffled likelihoods 0...
Loading pickled shuffled likelihoods 1...
Loading pickled shuffled likelihoods 2...
Loading pickled shuffled likelihoods 3...
Loading pickled shuffled likelihoods 4...
Loading pickled shuffled likelihoods 5...
Loading pickled shuffled likelihoods 6...
Loading pickled shuffled likelihoods 7...
Loading pickled shuffled likelihoods 8...
Loading pickled shuffled likelihoods 9...
Loading pickled shuffled likelihoods 10...
Loading pickled shuffled likelihoods 11...
Loading pickled shuffled likelihoods 12...
Loading pickled shuffled likelihoods 13...
Loading pickled shuffled likelihoods 14...
Loading pickled shuffled likelihoods 15...
Loading pickled shuffled likelihoods 16...
Loading pickled shuffled likelihoods 17...
Loading pickled shuffled likelihoods 18...
Loading pickled shuff

R068d1
Loading pickled true likelihoods...
Loading pickled shuffled likelihoods 0...
Loading pickled shuffled likelihoods 1...
Loading pickled shuffled likelihoods 2...
Loading pickled shuffled likelihoods 3...
Loading pickled shuffled likelihoods 4...
Loading pickled shuffled likelihoods 5...
Loading pickled shuffled likelihoods 6...
Loading pickled shuffled likelihoods 7...
Loading pickled shuffled likelihoods 8...
Loading pickled shuffled likelihoods 9...
Loading pickled shuffled likelihoods 10...
Loading pickled shuffled likelihoods 11...
Loading pickled shuffled likelihoods 12...
Loading pickled shuffled likelihoods 13...
Loading pickled shuffled likelihoods 14...
Loading pickled shuffled likelihoods 15...
Loading pickled shuffled likelihoods 16...
Loading pickled shuffled likelihoods 17...
Loading pickled shuffled likelihoods 18...
Loading pickled shuffled likelihoods 19...
Loading pickled shuffled likelihoods 20...
Loading pickled shuffled likelihoods 21...
Loading pickled shuff